In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, Concatenate, BatchNormalization, Activation
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
import random

# Define paths
data_dir = "cv_p3_images_split"
train_gray_dir = os.path.join(data_dir, "train/grayscale")
train_color_dir = os.path.join(data_dir, "train/colored")
val_gray_dir = os.path.join(data_dir, "validation/grayscale")
val_color_dir = os.path.join(data_dir, "validation/colored")

# Image dimensions
IMG_HEIGHT, IMG_WIDTH = 256, 256  # Resize all images to 256x256

# Utility to preprocess images
def preprocess_image(image_path, target_size):
    image = load_img(image_path, target_size=target_size, color_mode="rgb")
    image = img_to_array(image) / 255.0  # Normalize to [0, 1]
    return image

def load_images_from_folder(folder, target_size):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        images.append(preprocess_image(img_path, target_size))
    return np.array(images)


def load_dataset(gray_folder, color_folder, target_size, fraction=1.0):
    gray_images = []
    color_images = []
    
    filenames = os.listdir(gray_folder)
    if fraction < 1.0:
        filenames = random.sample(filenames, int(len(filenames) * fraction))  # Sample fraction of filenames
    
    for filename in filenames:
        gray_path = os.path.join(gray_folder, filename)
        color_path = os.path.join(color_folder, filename)
        gray_images.append(preprocess_image(gray_path, target_size)[..., 0:1])  # Extract grayscale channel
        color_images.append(preprocess_image(color_path, target_size))
    
    return np.array(gray_images), np.array(color_images)


train_gray, train_color = load_dataset(train_gray_dir, train_color_dir, (IMG_HEIGHT, IMG_WIDTH), 0.25)
val_gray, val_color = load_dataset(val_gray_dir, val_color_dir, (IMG_HEIGHT, IMG_WIDTH), 0.25)

# Define the colorization model
def build_model(loss="mse"):
    inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 1))

    # Encoder
    x = Conv2D(32, (3, 3), padding="same", strides=2)(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(64, (3, 3), padding="same", strides=2)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # Decoder
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = UpSampling2D((2, 2))(x)
    x = Conv2D(3, (3, 3), padding="same")(x)
    outputs = Activation("sigmoid")(x)

    return Model(inputs, outputs)

model_mse = build_model()
model_mse.compile(optimizer="adam", loss="mse", metrics=["accuracy"])
history = model_mse.fit(
    train_gray, train_color,
    validation_data=(val_gray, val_color),
    epochs=20,
    batch_size=16
)
model_mse.save("colorization_model_mse.h5")

model_mae = build_model(loss="mae")
model_mae.compile(optimizer="adam", loss="mae", metrics=["accuracy"])
history = model_mae.fit(
    train_gray, train_color,
    validation_data=(val_gray, val_color),
    epochs=20,
    batch_size=16
)
model_mae.save("colorization_model_mae.h5")

model_bce = build_model(loss="binary_crossentropy")
model_bce.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
history = model_bce.fit(
    train_gray, train_color,
    validation_data=(val_gray, val_color),
    epochs=20,
    batch_size=16
)
model_bce.save("colorization_model_bce.h5")

Epoch 1/20


In [ ]:
# Test the model and save results
test_gray_dir = os.path.join(data_dir, "test/grayscale")
test_color_dir = os.path.join(data_dir, "test/colored")
os.makedirs(test_color_dir, exist_ok=True)

def save_colorized_images(model, gray_dir, color_dir):
    for filename in os.listdir(gray_dir):
        gray_path = os.path.join(gray_dir, filename)
        gray_image = preprocess_image(gray_path, (IMG_HEIGHT, IMG_WIDTH))[..., 0:1]
        gray_image = np.expand_dims(gray_image, axis=0)  # Add batch dimension

        predicted_color = model.predict(gray_image)[0]
        predicted_color = np.clip(predicted_color * 255, 0, 255).astype(np.uint8)

        output_path = os.path.join(color_dir, filename)
        tf.keras.preprocessing.image.save_img(output_path, predicted_color)

save_colorized_images(model, test_gray_dir, test_color_dir)

print("Colorization complete. Output saved in:", test_color_dir)